#### Question:3

You are provided with the data of ratings of 36383 movies by 69324 movie watcher. The data in the movies.txt is in the form **movie id::movie name (year)::genre|genre|genre** . The data in the ratings.txt file is in the form user **id::movie id::rating::rating time stamp** . Write MapReduce routines to accomplish the following tasks.

3. Top ten movies by year; and

In [46]:
%%file mapq31.py
import sys

#for mapping (movie_id, movie_rating) from ratings.txt

for line in sys.stdin:
    data = line.strip().split('::')
    movie_id = data[1]
    movie_ratings = int(data[2])
    print("%s\t%s" %(movie_id, str(movie_ratings)))

Overwriting mapq31.py


In [47]:
!type ratings.txt | python mapq31.py | sort > out1.txt

In [43]:
%%file mapq32.py
import sys

#for mapping (movie_id, movie_name, movie_year) from movies.txt

for line in sys.stdin:
    data = line.strip().split("::")
    movie_id = data[0]
    movie_name = data[1][:-6]
    movie_yr = data[1][-5:-1]
    print("%s\t%s\t%s" % (movie_id, movie_name, movie_yr))   

Overwriting mapq32.py


In [44]:
!type movies.txt | python mapq32.py | sort > out2.txt

In [70]:
!type out1.txt out2.txt | sort /r  > finalout.txt


out1.txt



out2.txt




In [49]:
#can add one more map function for adding data points from the above two map functions

In [71]:
%%file supermapq3.py
import sys


ln = sys.stdin.readline().strip().split('\t')
curr_movie_id = ln[0]
curr_movie = ln[1]
curr_year = ln[2]

for line in sys.stdin:
    data = line.strip().split('\t')
    if len(data) == 2:
        movie_id = data[0]
        movie_rating = data[1]
        print("%s\t%s\t%s\t%s" % (curr_movie_id, curr_movie , curr_year, movie_rating))
        
    elif len(data) == 3:
        curr_movie_id = data[0]
        curr_movie = data[1]
        curr_year = data[2]    

Overwriting supermapq3.py


In [100]:
#!type finalout.txt | python supermapq3.py | sort 

In [75]:
%%file reduce3.py
import sys

initial = sys.stdin.readline().strip().split('\t')
curr_id = initial[0]
curr_name = initial[1]
curr_year = int(initial[2])
curr_sum = int(initial[3])
counter = 1

for line in sys.stdin:
    data = line.strip().split('\t')
    
    if data[0] == curr_id:
        counter +=1
        curr_sum += int(data[3])
        
    else:
        print("%s\t%s\t%s\t%s" % (curr_year, round(curr_sum/counter,3), curr_id, curr_name))
        curr_id = data[0]
        curr_name = data[1]
        curr_year = int(data[2])
        curr_sum = int(data[3])
        counter = 1

Overwriting reduce3.py


In [79]:
!type finalout.txt | python supermapq3.py | sort | python reduce3.py | sort /r > finalmineddata.txt

In [98]:
%%file reducefinalq3.py
import sys

initial = sys.stdin.readline().strip().split('\t')
curr_year = int(initial[0])
curr_rating = initial[1]
curr_id = initial[2]
curr_name = initial[3]
counter = 1

for line in sys.stdin:
    data = line.strip().split('\t')
    #print(data)
    yr = int(data[0])
    rating = data[1]
    id_ = data[2]
    name = data[3]
    
    if int(curr_year) == int(yr) and counter <= 10:
        #print(counter)
        print("%s\t%s\t%s\t%s" % (yr, rating , id_, name))
        counter +=1
    
    elif int(curr_year) == int(yr) and counter > 10:
        counter +=1
        pass
        
    else:
        print("%s\t%s\t%s\t%s" % (curr_year, curr_rating , curr_id, curr_name))
        counter = 1
        curr_year = yr
        curr_rating = rating
        curr_id = id_
        curr_name = name
        
        

Overwriting reducefinalq3.py


In [99]:
!type finalmineddata.txt | python reducefinalq3.py

2021	10.0	5537002	Killers of the Flower Moon
2020	9.5	10948274	Gemmeker
2020	9.333	10370380	The Windermere Children
2020	9.061	8503618	Hamilton
2020	9.0	9537292	Ala Vaikunthapurramuloo
2020	9.0	6393674	Before/During/After
2020	9.0	3197274	Showbiz Kids
2020	9.0	12498618	8:46
2020	9.0	12058882	Coachella: 20 Years in the Desert
2020	9.0	11916278	The Scheme
2020	9.0	11860624	Michael McIntyre: Showman
2020	9.5	11454066	You Cannot Kill David Arquette
2019	9.857	9536832	Kitbull
2019	9.667	8091892	Gekijouban Fate/Stay Night: Heaven's Feel - II. Lost Butterfly
2019	9.667	10011448	Linda Ronstadt: The Sound of My Voice
2019	9.6	10472884	Seishun Buta Yaro wa Yumemiru Shoujo no Yume wo Minai
2019	9.5	5593384	TÃµde ja Ãµigus
2019	9.333	10883124	Dads
2019	9.333	10375886	Toni Kroos
2019	9.25	8902990	The Sky Is Pink
2019	9.088	7286456	Joker
2019	9.071	10847306	Bill Burr: Paper Tiger
2019	9.989	8579674	1917
2018	9.5	7467476	Fauve
2018	9.5	6776106	AndiÃ° eÃ°lilega
2018	9.318	9095324	Coldplay: A Head Full